In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from keras.applications import InceptionResNetV2
inc_conv = InceptionResNetV2(weights='imagenet',
                             include_top=False,
                             input_shape=(256, 256, 3)
                             )

In [15]:
for layer in inc_conv.layers[:-4]:
  layer.trainable = False
for layer in inc_conv.layers:
  print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fb5de6b7c88> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb5de6b7a20> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb5de66ebe0> False
<tensorflow.python.keras.layers.core.Activation object at 0x7fb5de688e10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb5de61de10> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb5de688e80> False
<tensorflow.python.keras.layers.core.Activation object at 0x7fb5de629f60> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb5de637470> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb5de62fac8> False
<tensorflow.python.keras.layers.core.Activation object at 0x7fb5de6298d0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fb5de83ff28> False
<tensorflow.python.keras.layers.c

In [16]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(inc_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(34, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
flatten (Flatten)            (None, 55296)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              56624128  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 34)                34850     
Total params: 110,995,714
Trainable params: 59,855,394
Non-trainable params: 51,140,320
_________________________________________________________________


In [17]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/ok')
zip_ref.close()

In [18]:
base_dir = '/content/ok'

train_dir = os.path.join(base_dir, 'imagedb')
print(train_dir)

/content/ok/imagedb


In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
shift = 0.03
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.2,
                                   rotation_range=90,
                                   zoom_range=0.3,
                                   width_shift_range=shift,
                                   height_shift_range=shift,
                                   horizontal_flip=True,
                                   vertical_flip=True

                                   )

val_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir,batch_size = 80,
                                                    class_mode = 'categorical',
                                                    target_size=(256,256),
                                                    #color_mode='grayscale',
                                                    shuffle = True,
                                                    subset = 'training',
                                                    seed=4
                                                    
                                                    )
validation_generator = val_datagen.flow_from_directory(train_dir,
                                                    batch_size = 80,
                                                    class_mode = 'categorical',
                                                    #color_mode='grayscale',
                                                    target_size=(256,256),
                                                    shuffle = True,
                                                    subset = 'validation',
                                                    seed=4
                                                    )


Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [25]:
import datetime
import tensorflow as tf
callbacks = []
save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)
model.compile(loss= 'categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics = ['acc'])
history = model.fit(train_generator,
                    steps_per_epoch = train_generator.samples/train_generator.batch_size,
                    epochs = 30,
                    validation_data = validation_generator,
                    verbose =1,
                    callbacks=callbacks
                    )
model.save('small;_last4.h5')

Epoch 1/30
30/30 [==============================] - 76s 2s/step - loss: 0.5080 - acc: 0.8564 - val_loss: 0.2537 - val_acc: 0.9349

Epoch 00001: val_loss improved from inf to 0.25365, saving model to best_weights.hdf5
Epoch 2/30
30/30 [==============================] - 60s 2s/step - loss: 0.3704 - acc: 0.8902 - val_loss: 0.2691 - val_acc: 0.9232

Epoch 00002: val_loss did not improve from 0.25365
Epoch 3/30
30/30 [==============================] - 60s 2s/step - loss: 0.2535 - acc: 0.9146 - val_loss: 0.2471 - val_acc: 0.9199

Epoch 00003: val_loss improved from 0.25365 to 0.24711, saving model to best_weights.hdf5
Epoch 4/30
30/30 [==============================] - 59s 2s/step - loss: 0.2488 - acc: 0.9199 - val_loss: 0.2352 - val_acc: 0.9332

Epoch 00004: val_loss improved from 0.24711 to 0.23517, saving model to best_weights.hdf5
Epoch 5/30
30/30 [==============================] - 59s 2s/step - loss: 0.2068 - acc: 0.9367 - val_loss: 0.2080 - val_acc: 0.9366

Epoch 00005: val_loss improv

In [26]:
test_datagen  = ImageDataGenerator(rescale=1./255)
test_dir = os.path.join(base_dir, 'imagedb_test')
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=10,
                                                         class_mode  = 'categorical',
                                                         
                                                         target_size=(256,256)
                                                         ) 
loss, acc = model.evaluate(test_generator)

Found 2149 images belonging to 34 classes.
215/215 [==============================] - 48s 221ms/step - loss: 0.3651 - acc: 0.8902
